#### Import

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark.version)

23/06/15 03:16:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/15 03:16:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/15 03:16:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/15 03:16:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/06/15 03:16:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


3.1.3


In [2]:
from Student import Student, czytaj, ileElementow

lines = spark.sparkContext.textFile("studenciOU.txt")
dane = lines.map(lambda line: Student(line.split(",")))

print("Ilość danych w dokumencie: \n", dane.count())
print("Pierwsza linia dokumentu: \n", dane.first())

Ilość danych w dokumencie: 
 32593
Pierwsza linia dokumentu: 
 {'kod': 'AAA', 'id': '11391', 'plec': 'M', 'region': 'East Anglian Region', 'wyksztalcenie': 'HE Qualification', 'wiek': 68, 'prob': 0, 'punktow': 240, 'zaliczenie': 'Pass'}


### Zadanie B (strumienie i kolekcje Spark)

#### 1. Dane z kodem zaczynającym się od E.

In [23]:
for osoba in dane.filter(lambda osoba: osoba.kod.startswith("E")).take(5):
    print(osoba)   

{'kod': 'EEE', 'id': '26915', 'plec': 'M', 'region': 'Wales', 'wyksztalcenie': 'Lower Than A Level', 'wiek': 44, 'prob': 0, 'punktow': 30, 'zaliczenie': 'Fail'}
{'kod': 'EEE', 'id': '32553', 'plec': 'M', 'region': 'North Western Region', 'wyksztalcenie': 'A Level or Equivalent', 'wiek': 23, 'prob': 0, 'punktow': 60, 'zaliczenie': 'Withdrawn'}
{'kod': 'EEE', 'id': '35355', 'plec': 'M', 'region': 'Wales', 'wyksztalcenie': 'Lower Than A Level', 'wiek': 51, 'prob': 0, 'punktow': 30, 'zaliczenie': 'Pass'}
{'kod': 'EEE', 'id': '40508', 'plec': 'M', 'region': 'West Midlands Region', 'wyksztalcenie': 'A Level or Equivalent', 'wiek': 41, 'prob': 0, 'punktow': 90, 'zaliczenie': 'Withdrawn'}
{'kod': 'EEE', 'id': '42638', 'plec': 'M', 'region': 'Scotland', 'wyksztalcenie': 'HE Qualification', 'wiek': 50, 'prob': 0, 'punktow': 60, 'zaliczenie': 'Pass'}


#### 2. Liczbę osób, które próbowały zdać wcześniej jeden raz (z jedną próbą).

In [4]:
print("Liczba osób, które próbowały zdać z JEDNĄ próbą:", len(dane.filter(lambda osoba: osoba.prob == 1).collect()))

Liczba osób, które próbowały zdać z JEDNĄ próbą: 3299


#### 3. Liczbę osób w wieku 25-35 lat.

In [5]:
print("Liczba osób, w wieku 25-35 lat:", len(dane.filter(lambda osoba: osoba.wiek >= 25 and osoba.wiek <= 35).collect()))

Liczba osób, w wieku 25-35 lat: 11808


#### 4. Listę nazw wszystkich regionów występujących w zestawieniu (bez powtórzeń).

In [27]:
print("Lista nazw wszystkich regionów (bez powtórzeń):")
for region in dane.map(lambda student: student.region).distinct().collect():
    print(region)

Lista nazw wszystkich regionów (bez powtórzeń):
East Anglian Region
Scotland
South East Region
West Midlands Region
North Region
Ireland
South West Region
East Midlands Region
London Region
North Western Region
Wales
South Region
Yorkshire Region


#### 5. Liczbę osób każdej z płci.

In [36]:
#d = dane.groupBy("plec").map(lambda gr: (gr[0], len(gr[1]))).collect()
#
#print("Liczba osób dla każdej z płci:")
#for gender, count in d:
#    print(gender, count)
    
ls8 = dane.map(lambda x: (x.plec, 1)).reduceByKey(lambda a, b: a + b).collect()
print(*ls8, sep="\n")

('M', 17875)
('F', 14718)


#### 6. Średni wiek osób z Walii (Wales)

In [39]:
d = dane.filter(lambda el: el.region=="Wales").map(lambda el: el.wiek)
print(d.sum()/d.count())

30.748322147651006


#### 7. Wiek najstarszego mężczyzny który zdał

In [41]:
d = dane.filter(lambda el: el.plec=="M").map(lambda el: el.wiek)
print(d.max())

74


#### 8. Liczbę osób z regionu Londynu (London Region), które oblały dla każdej liczby punktów.

In [57]:
d = dane.filter(lambda o: o.region == "London Region" and osoba.zaliczenie == "Pass").collect()
print(len(d))

3216


#### 9. Dane osób, które otrzymały mniej niż 50 punktów.

In [58]:
ls1 = dane.filter(lambda o: o.punktow < 50).take(5)
print(*ls1, sep="\n")

{'kod': 'CCC', 'id': '28418', 'plec': 'F', 'region': 'West Midlands Region', 'wyksztalcenie': 'A Level or Equivalent', 'wiek': 22, 'prob': 0, 'punktow': 30, 'zaliczenie': 'Fail'}
{'kod': 'CCC', 'id': '40333', 'plec': 'M', 'region': 'North Region', 'wyksztalcenie': 'HE Qualification', 'wiek': 44, 'prob': 0, 'punktow': 30, 'zaliczenie': 'Withdrawn'}
{'kod': 'CCC', 'id': '40604', 'plec': 'M', 'region': 'Ireland', 'wyksztalcenie': 'A Level or Equivalent', 'wiek': 49, 'prob': 0, 'punktow': 30, 'zaliczenie': 'Pass'}
{'kod': 'CCC', 'id': '59541', 'plec': 'M', 'region': 'East Anglian Region', 'wyksztalcenie': 'A Level or Equivalent', 'wiek': 25, 'prob': 0, 'punktow': 30, 'zaliczenie': 'Withdrawn'}
{'kod': 'CCC', 'id': '62278', 'plec': 'M', 'region': 'West Midlands Region', 'wyksztalcenie': 'A Level or Equivalent', 'wiek': 22, 'prob': 0, 'punktow': 40, 'zaliczenie': 'Fail'}


#### 10. Liczbę osób w wieku do 23 lat z South Region.

In [60]:
print(dane.filter(lambda x: x.wiek < 23 and x.region == "South Region").count())


850


#### 11. Listę wszystkie rodzajów wykształcenia występujących w danych (HE Qualification, Lower Than A Level itd.) (bez powtórzeń)

In [61]:
for wyk in dane.map(lambda os: os.wyksztalcenie).distinct().collect():
    print(wyk)


HE Qualification
A Level or Equivalent
Lower Than A Level
Post Graduate Qualification
No Formal quals


#### 12. Liczbę osób dla każdego kodu

In [51]:
ls5 = dane.map(lambda x: (x.kod, 1)).reduceByKey(lambda a, b: a + b).collect()
print(*ls5, sep="\n")

('AAA', 748)
('BBB', 7909)
('CCC', 4434)
('EEE', 2934)
('DDD', 6272)
('FFF', 7762)
('GGG', 2534)


#### 13. Średnią liczbę punktów osób które zdały

In [50]:
ls6 = dane.filter(lambda x: x.zaliczenie == 'Pass').map(lambda x: x.punktow).mean()
print(ls6)

74.22093681740941


#### 14. Najmniejszą liczbę prób dla osób ze Szkocji (rejon Scotland)

In [49]:
ls7 = dane.filter(lambda x: x.region == 'Scotland').map(lambda x: x.prob).min()
print(ls7)


0


#### 15. Liczbę osób w wieku powyżej 40 lat które zdawały co najmniej raz wcześniej dla każdego regionu.

In [48]:
ls8 = dane.filter(lambda x: x.wiek > 40 and x.prob >=1 ).map(lambda x: (x.region, 1)).reduceByKey(lambda a, b: a + b).collect()
print(*ls8, sep="\n")

('East Midlands Region', 44)
('South East Region', 63)
('East Anglian Region', 62)
('North Region', 49)
('South West Region', 74)
('London Region', 109)
('Scotland', 102)
('Ireland', 37)
('West Midlands Region', 71)
('South Region', 88)
('Yorkshire Region', 46)
('Wales', 67)
('North Western Region', 82)
